In [ ]:
import numpy as np
import pandas as pd
import os

import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['font.family']='NanumBarunGothic'
%matplotlib inline
from pylab import rcParams
rcParams['figure.figsize'] = (12,5)

# 관련 라이브러리 임포트
import matplotlib.font_manager as fm

# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
matplotlib.rcParams['axes.unicode_minus'] = False

In [ ]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tools.eval_measures import mse, rmse, aic
from statsmodels.tsa.stattools import grangercausalitytests

from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

import warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 전처리한 데이터 불러오기
filepath='/content/drive/Shareddrives/팔당댐/combined_data/'
df=pd.read_csv(filepath+'filled_data(2012-2022).csv', index_col='ymdhm', parse_dates=['ymdhm'])
df

,swl,inf,sfw,ecpc,tototf,tide_level,wl_1018662,fw_1018662,wl_1018680,fw_1018680,wl_1018683,fw_1018683,wl_1019630,fw_1019630,rf_10184100,rf_10184110,rf_10184140
ymdhm,,,,,,,,,,,,,,,,,
2012-05-01 00:00:00,24.800,555.00,219.07,24.93,555.00,445.0,310.7,469.05,300.2,0.0,290.0,729.80,275.3,540.18,0.0,0.0,0.0
2012-05-01 00:10:00,24.794,464.60,218.86,25.15,562.90,449.0,314.7,498.00,300.2,0.0,290.0,731.48,275.3,540.18,0.0,0.0,0.0
2012-05-01 00:20:00,24.789,478.10,218.69,25.31,576.40,451.0,313.7,490.68,301.2,0.0,290.0,726.42,275.3,540.18,0.0,0.0,0.0
2012-05-01 00:30:00,24.789,464.80,218.69,25.31,563.10,452.0,311.7,476.21,301.2,0.0,290.0,726.42,276.3,552.17,0.0,0.0,0.0
2012-05-01 00:40:00,24.789,478.10,218.69,25.31,576.40,450.0,311.7,476.21,301.2,0.0,291.0,707.17,277.3,564.29,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-18 23:10:00,25.040,259.23,212.86,31.14,259.23,510.0,0.0,319.84,0.0,0.0,0.0,-456.41,0.0,974.40,0.0,0.0,0.0
2022-07-18 23:20:00,25.040,260.46,212.86,31.14,260.46,492.0,0.0,314.01,0.0,0.0,0.0,-717.30,0.0,1006.88,0.0,0.0,0.0
2022-07-18 23:30:00,25.040,259.37,212.86,31.14,259.37,475.0,0.0,387.55,0.0,0.0,0.0,-843.37,0.0,1039.90,0.0,0.0,0.0


In [ ]:
# fw_1018680는 원래 없는 자료이므로 삭제
df.drop('fw_1018680', axis=1, inplace=True)

# 변수 적합성 파악
- Granger Causality
- 다중공선성(VIF)

## Granger Causality

In [ ]:
def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False, maxlag=5):

    """Check Granger Causality of all possible combinations of the Time series.
    The rows are the response variable, columns are predictors.

    data      : pandas dataframe containing the time series variables
    variables : list containing names of the time series variables.
    """
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)

            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df

variables_list=['wl_1018662', 'wl_1018680', 'wl_1018683', 'wl_1019630',
                'swl','inf','sfw','ecpc','tototf', 'tide_level',
                'fw_1018662', 'fw_1018683', 'fw_1019630','rf_10184100', 'rf_10184110','rf_10184140']

grangers_causation_matrix(df, variables=variables_list)

,wl_1018662_x,wl_1018680_x,wl_1018683_x,wl_1019630_x,swl_x,inf_x,sfw_x,ecpc_x,tototf_x,tide_level_x,fw_1018662_x,fw_1018683_x,fw_1019630_x,rf_10184100_x,rf_10184110_x,rf_10184140_x
wl_1018662_y,1.0,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0000,0.0,0.0000,0.0,0.0000,0.0000,0.0000,0.000,0.0000
wl_1018680_y,0.0,1.0,0.0,0.0000,0.0,0.0,0.0000,0.0000,0.0,0.0000,0.0,0.0000,0.0000,0.0000,0.000,0.0000
wl_1018683_y,0.0,0.0,1.0,0.0000,0.0,0.0,0.0000,0.0000,0.0,0.0000,0.0,0.0000,0.0000,0.0000,0.000,0.0000
wl_1019630_y,0.0,0.0,0.0,1.0000,0.0,0.0,0.0000,0.0000,0.0,0.0000,0.0,0.0000,0.0000,0.0000,0.000,0.0000
swl_y,0.0,0.0,0.0,0.0000,1.0,0.0,0.0000,0.0000,0.0,0.0055,0.0,0.0000,0.0000,0.0000,0.000,0.0001
inf_y,0.0,0.0,0.0,0.0000,0.0,1.0,0.0000,0.0000,0.0,0.0023,0.0,0.0000,0.0000,0.0000,0.000,0.0000
sfw_y,0.0,0.0,0.0,0.0014,0.0,0.0,0.9999,0.0002,0.0,0.0206,0.0,0.0289,0.0011,0.0001,0.000,0.0000
ecpc_y,0.0,0.0,0.0,0.0014,0.0,0.0,0.0000,1.0000,0.0,0.0205,0.0,0.0284,0.0011,0.0001,0.000,0.0000
tototf_y,0.0,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0000,1.0,0.0992,0.0,0.0000,0.0000,0.0000,0.000,0.0000
tide_level_y,0.0,0.0,0.0,0.0000,0.0,0.0,0.0877,0.0889,0.0,1.0000,0.0,0.0000,0.0000,0.0875,0.015,0.0009


## 다중공선성(VIF)

In [ ]:
def feature_engineering_XbyVIF(X_train):
    vif = pd.DataFrame()
    vif['VIF_Factor'] = [variance_inflation_factor(X_train.values, i)
                         for i in range(X_train.shape[1])]
    vif['Feature'] = X_train.columns
    return vif

vif=feature_engineering_XbyVIF(df.drop('wl_1018662',axis=1))
print(vif)

      VIF_Factor      Feature
0   76073.974086          swl
1      41.378578          inf
2   63106.513880          sfw
3     748.827257         ecpc
4      47.093257       tototf
5       5.629266   tide_level
6      17.815411   fw_1018662
7    3292.673677   wl_1018680
8    5443.046576   wl_1018683
9       7.916823   fw_1018683
10   1064.306938   wl_1019630
11     14.149894   fw_1019630
12      1.279141  rf_10184100
13      1.318648  rf_10184110
14      1.401597  rf_10184140


VIF가 10 이상이면 다중공선성이 있다고 해석 <br>
=> 대부분의 변수들에서 다중공선성이 높음 <br>
=> 일부 변수 제거하기에는 다중공선성이 있는 변수가 너무 많으므로 PCA 통해서 차원축소

# train-test
- trainset : 2012~2021 => CV => model
- new_trainset : 2022-05-01 ~ 2022-05-31
- new_testset : 2022-06-01 ~ 2022-07-18

In [ ]:
y_cols=['wl_1018662', 'wl_1018680', 'wl_1018683', 'wl_1019630']
df_X=df.drop(y_cols,axis=1)
df_y=df[y_cols]

# 각 종속변수 별 2012~2021 학습데이터
def X_train_for_y1(df_X, df_y, y1):
    df_y234=df_y.drop(y1,axis=1)
    X_train=pd.merge(df_X.loc[:'2021'],df_y234.loc[:'2021'],on='ymdhm')
    return X_train

# 각 종속변수 별 2022년 5월 학습데이터
def X_newTrain_for_y1(df_X, df_y, y1):
    df_y234=df_y.drop(y1,axis=1)
    X_newTrain=pd.merge(df_X.loc['2022-05'],df_y234.loc['2022-05'],on='ymdhm')
    return X_newTrain

# 각 종속변수 별 2022년 6~7월 테스트데이터
def X_newTest_for_y1(df_X, df_y, y1):
    df_y234=df_y.drop(y1,axis=1)
    X_newTest=pd.merge(df_X.loc['2022-06':],df_y234.loc['2022-06':],on='ymdhm')
    return X_newTest


In [ ]:
X_train_y1=X_train_for_y1(df_X, df_y, 'wl_1018662')
X_newTrain_y1=X_newTrain_for_y1(df_X, df_y, 'wl_1018662')
X_newTest_y1=X_newTest_for_y1(df_X, df_y, 'wl_1018662')

X_train_y2=X_train_for_y1(df_X, df_y, 'wl_1018680')
X_newTrain_y2=X_newTrain_for_y1(df_X, df_y, 'wl_1018680')
X_newTest_y2=X_newTest_for_y1(df_X, df_y, 'wl_1018680')

X_train_y3=X_train_for_y1(df_X, df_y, 'wl_1018683')
X_newTrain_y3=X_newTrain_for_y1(df_X, df_y, 'wl_1018683')
X_newTest_y3=X_newTest_for_y1(df_X, df_y, 'wl_1018683')

X_train_y4=X_train_for_y1(df_X, df_y, 'wl_1019630')
X_newTrain_y4=X_newTrain_for_y1(df_X, df_y, 'wl_1019630')
X_newTest_y4=X_newTest_for_y1(df_X, df_y, 'wl_1019630')

# 스케일링

- 종속변수 1개 제외한 나머지 X 변수들 => scaling  & PCA
- 종속변수 4개 => scaling

In [ ]:
def scale_X(X_train, X_newTrain, X_newTest):
    scaler=MinMaxScaler()
    sc_Xtrain=scaler.fit_transform(X_train)
    sc_X_newTrain=scaler.transform(X_newTrain)
    sc_X_newTest=scaler.transform(X_newTest)
    Xtrain=pd.DataFrame(sc_Xtrain, columns=X_train.columns, index=X_train.index)
    XnewTrain=pd.DataFrame(sc_X_newTrain, columns=X_newTrain.columns, index=X_newTrain.index)
    XnewTest=pd.DataFrame(sc_X_newTest, columns=X_newTest.columns, index=X_newTest.index)
    return scaler, sc_Xtrain, sc_X_newTrain, sc_X_newTest, Xtrain, XnewTrain, XnewTest

def scale_y(y_train, y_newTrain, y_newTest, y_dap):
    scaler=MinMaxScaler()
    sc_ytrain=scaler.fit_transform(y_train)
    sc_y_newTrain=scaler.transform(y_newTrain)
    sc_y_newTest=scaler.transform(y_newTest)
    sc_y_dap=scaler.transform(y_dap)
    yTrain=pd.DataFrame(sc_ytrain, columns=y_train.columns, index=y_train.index)
    ynewTrain=pd.DataFrame(sc_y_newTrain, columns=y_newTrain.columns, index=y_newTrain.index)
    ynewTest=pd.DataFrame(sc_y_newTest, columns=y_newTest.columns, index=y_newTest.index)
    yDap=pd.DataFrame(sc_y_dap, columns=y_dap.columns, index=y_dap.index)
    return scaler, sc_ytrain, sc_y_newTrain, sc_y_newTest, sc_y_dap, yTrain, ynewTrain, ynewTest, yDap

In [ ]:
Xscaler_y1, scX_train_y1, scX_newTrain_y1, scX_newTest_y1, Xtrain_y1, XnewTrain_y1, XnewTest_y1 = scale_X(X_train_y1, X_newTrain_y1, X_newTest_y1)
scX_train_y1

array([[0.55425824, 0.05084145, 0.65205438, ..., 0.06      , 0.05162065,
        0.05647841],
       [0.55013736, 0.04746819, 0.6488468 , ..., 0.06      , 0.05162065,
        0.05647841],
       [0.5467033 , 0.04797194, 0.64625019, ..., 0.06111111, 0.05162065,
        0.05647841],
       ...,
       [0.71909341, 0.03222135, 0.55720177, ..., 0.01333333, 0.02160864,
        0.02159468],
       [0.71909341, 0.03222135, 0.55720177, ..., 0.01333333, 0.02160864,
        0.02159468],
       [0.71909341, 0.03222135, 0.55720177, ..., 0.01333333, 0.02160864,
        0.02159468]])

In [ ]:
Xscaler_y2, scX_train_y2, scX_newTrain_y2, scX_newTest_y2, Xtrain_y2, XnewTrain_y2, XnewTest_y2 = scale_X(X_train_y2, X_newTrain_y2, X_newTest_y2)
Xscaler_y3, scX_train_y3, scX_newTrain_y3, scX_newTest_y3, Xtrain_y3, XnewTrain_y3, XnewTest_y3 = scale_X(X_train_y3, X_newTrain_y3, X_newTest_y3)
Xscaler_y4, scX_train_y4, scX_newTrain_y4, scX_newTest_y4, Xtrain_y4, XnewTrain_y4, XnewTest_y4 = scale_X(X_train_y4, X_newTrain_y4, X_newTest_y4)

In [ ]:
y_cols=['wl_1018662', 'wl_1018680', 'wl_1018683', 'wl_1019630']
ys_train=df_y[y_cols].loc[:'2021']
ys_newTrain=df_y[y_cols].loc['2022-05']
ys_newTest=df_y[y_cols].loc['2022-06':]

In [ ]:
# 종속변수 testset 정답
df_dap=pd.read_csv(filepath+'combined_train_test_filled_cleansed.csv', index_col='ymdhm', parse_dates=['ymdhm'])
ys_dap=df_dap[y_cols].loc['2022-06':]

In [ ]:
y_scaler, sc_ys_train, sc_ys_newTrain, sc_ys_newTest, sc_ys_dap, ysTrain, ysNewTrain, ysNewTest, ysDap = scale_y(ys_train, ys_newTrain, ys_newTest, ys_dap)
sc_ys_train

array([[0.04961089, 0.06      , 0.05162065, 0.05647841],
       [0.05350195, 0.06      , 0.05162065, 0.05647841],
       [0.05252918, 0.06111111, 0.05162065, 0.05647841],
       ...,
       [0.01070039, 0.01333333, 0.02160864, 0.02159468],
       [0.01167315, 0.01333333, 0.02160864, 0.02159468],
       [0.01070039, 0.01333333, 0.02160864, 0.02159468]])

# PCA
- EDA에서 ['swl','sfw','ecpc'],['sfw','inf','tototf']이 서로 연관됨을 알았기 때문에 ['wl'],['fw'],['rf']까지 총 5개의 차원으로 축소할 수 있을 것으로 예상
- n_components=5

In [ ]:
def get_pc_df(n, sc_X, df_ys, y_col):
    # pca 객체 생성
    pca=PCA(n_components=n)
    # 차원 축소후 변수에 저장
    pca_X_y=pca.fit_transform(sc_X) # (행수, n_components )
    # PCA 작업 후 데이타프레임에 추가
    df_pc=pd.DataFrame(index=df_ys.index)
    for i in range(n):
        col='pc'+str(i+1)
        df_pc[col]=pca_X_y[:,i]
    # y 종속변수와 merge
    df=pd.merge(df_ys[[y_col]], df_pc, on='ymdhm')
    return df

In [ ]:
# 종속변수1 - train
df_y1_pc_train=get_pc_df(5, scX_train_y1, ysTrain,'wl_1018662')
df_y1_pc_train

,wl_1018662,pc1,pc2,pc3,pc4,pc5
ymdhm,,,,,,
2012-05-01 00:00:00,0.049611,0.063450,0.112593,-0.079246,0.183270,0.009301
2012-05-01 00:10:00,0.053502,0.069717,0.116839,-0.080829,0.185339,0.009380
2012-05-01 00:20:00,0.052529,0.074120,0.118762,-0.082764,0.187366,0.008906
2012-05-01 00:30:00,0.050584,0.074094,0.120018,-0.083169,0.187208,0.007078
2012-05-01 00:40:00,0.050584,0.074437,0.117617,-0.081816,0.186938,0.005742
...,...,...,...,...,...,...
2021-10-31 23:10:00,0.009728,0.041697,-0.347734,-0.151622,-0.007342,0.038536
2021-10-31 23:20:00,0.010700,0.042046,-0.347804,-0.150895,-0.007532,0.037653
2021-10-31 23:30:00,0.010700,0.042066,-0.347922,-0.150823,-0.007519,0.037794


In [ ]:
# 종속변수2,3,4 - train
df_y2_pc_train=get_pc_df(5, scX_train_y2, ysTrain,'wl_1018680')
df_y3_pc_train=get_pc_df(5, scX_train_y3, ysTrain,'wl_1018683')
df_y4_pc_train=get_pc_df(5, scX_train_y4, ysTrain,'wl_1019630')

# newTrain
df_y1_pc_newTrain=get_pc_df(5, scX_newTrain_y1, ysNewTrain,'wl_1018662')
df_y2_pc_newTrain=get_pc_df(5, scX_newTrain_y2, ysNewTrain,'wl_1018680')
df_y3_pc_newTrain=get_pc_df(5, scX_newTrain_y3, ysNewTrain,'wl_1018683')
df_y4_pc_newTrain=get_pc_df(5, scX_newTrain_y4, ysNewTrain,'wl_1019630')

# newTest
df_y1_pc_newTest=get_pc_df(5, scX_newTest_y1, ysNewTest,'wl_1018662')
df_y2_pc_newTest=get_pc_df(5, scX_newTest_y2, ysNewTest,'wl_1018680')
df_y3_pc_newTest=get_pc_df(5, scX_newTest_y3, ysNewTest,'wl_1018683')
df_y4_pc_newTest=get_pc_df(5, scX_newTest_y4, ysNewTest,'wl_1019630')

# newTest+yDap
df_y1_pc_dap=get_pc_df(5, scX_newTest_y1, ysDap,'wl_1018662')
df_y2_pc_dap=get_pc_df(5, scX_newTest_y2, ysDap,'wl_1018680')
df_y3_pc_dap=get_pc_df(5, scX_newTest_y3, ysDap,'wl_1018683')
df_y4_pc_dap=get_pc_df(5, scX_newTest_y4, ysDap,'wl_1019630')

# 시계열 데이터 정상성 확인

In [ ]:
def adfuller_test(series, signif=0.05, name='', verbose=False):
    """Perform ADFuller to test for Stationarity of given series and print report"""
    r = adfuller(series, autolag='AIC')
    output = {'test_statistic':round(r[0], 4), 'pvalue':round(r[1], 4), 'n_lags':round(r[2], 4), 'n_obs':r[3]}
    p_value = output['pvalue']
    def adjust(val, length= 6): return str(val).ljust(length)

    # Print Summary
    print(f'    Augmented Dickey-Fuller Test on "{name}"', "\n   ", '-'*47)
    print(f' Null Hypothesis: Data has unit root. Non-Stationary.')
    print(f' Significance Level    = {signif}')
    print(f' Test Statistic        = {output["test_statistic"]}')
    print(f' No. Lags Chosen       = {output["n_lags"]}')

    for key,val in r[4].items():
        print(f' Critical value {adjust(key)} = {round(val, 3)}')

    if p_value <= signif:
        print(f" => P-Value = {p_value}. Rejecting Null Hypothesis.")
        print(f" => Series is Stationary.")
    else:
        print(f" => P-Value = {p_value}. Weak evidence to reject the Null Hypothesis.")
        print(f" => Series is Non-Stationary.")

In [ ]:
# # trainset-y1
# for name, column in df_y1_pc_train.iteritems():
#     adfuller_test(column, name=column.name)
#     print('\n')

In [ ]:
# # trainset-y2
# for name, column in df_y2_pc_train.iteritems():
#     adfuller_test(column, name=column.name)
#     print('\n')

In [ ]:
# # trainset-y3
# for name, column in df_y3_pc_train.iteritems():
#     adfuller_test(column, name=column.name)
#     print('\n')

In [ ]:
# # trainset-y4
# for name, column in df_y4_pc_train.iteritems():
#     adfuller_test(column, name=column.name)
#     print('\n')

In [ ]:
# # newTrain set-y1
# for name, column in df_y1_pc_newTrain.iteritems():
#     adfuller_test(column, name=column.name)
#     print('\n')

In [ ]:
# # newTrain set-y2
# for name, column in df_y2_pc_newTrain.iteritems():
#     adfuller_test(column, name=column.name)
#     print('\n')

In [ ]:
# # newTrain set-y3
# for name, column in df_y3_pc_newTrain.iteritems():
#     adfuller_test(column, name=column.name)
#     print('\n')

In [ ]:
# # newTrain set-y4
# for name, column in df_y4_pc_newTrain.iteritems():
#     adfuller_test(column, name=column.name)
#     print('\n')

In [ ]:
# # newTest set-y1
# for name, column in df_y1_pc_newTest.iteritems():
#     adfuller_test(column, name=column.name)
#     print('\n')

In [ ]:
# # newTest set-y2
# for name, column in df_y2_pc_newTest.iteritems():
#     adfuller_test(column, name=column.name)
#     print('\n')

In [ ]:
# # newTest set-y3
# for name, column in df_y3_pc_newTest.iteritems():
#     adfuller_test(column, name=column.name)
#     print('\n')

In [ ]:
# # newTest set-y4
# for name, column in df_y4_pc_newTest.iteritems():
#     adfuller_test(column, name=column.name)
#     print('\n')

# 최적 order p 찾기

# 모델링

## var(11)

In [ ]:
  df_tt_y1

,wl_1018662,pc1,pc2,pc3,pc4,pc5
ymdhm,,,,,,
2022-05-01 00:00:00,0.030156,-0.312020,-0.030539,0.047028,0.010046,0.018064
2022-05-01 00:10:00,0.031128,-0.323297,-0.032927,0.043751,0.003403,-0.004586
2022-05-01 00:20:00,0.030156,-0.333279,-0.034806,0.041785,-0.000028,-0.004446
2022-05-01 00:30:00,0.029183,-0.340770,-0.035834,0.040641,-0.002354,-0.004377
2022-05-01 00:40:00,0.027237,-0.340651,-0.028003,0.033292,-0.002429,-0.008842
...,...,...,...,...,...,...
2022-07-18 23:10:00,0.027529,0.123182,-0.172778,0.010886,-0.006156,-0.015705
2022-07-18 23:20:00,0.026556,0.102676,-0.164035,0.009340,-0.006746,-0.022059
2022-07-18 23:30:00,0.038230,0.086847,-0.149238,0.012111,-0.007474,-0.028260


In [ ]:
# 모델 학습 및 y 예측값 df 입력
def get_fc_ys(lag, df_train, df_tt, df_fc):
    # 모델 학습
    lag=lag
    var=VAR(df_train)
    fitted_var=var.fit(lag)
    for i in range(6912):
      # df_tt에서 예측하려는 시간 직전 lag개 데이터 가져오기
      forecast_input=df_tt.iloc[:4464+i,:].values[-lag:] # 4463행(2022-05-31 23:50:00)
      # 바로 다음 step 1개 예측
      fc=fitted_var.forecast(y=forecast_input, steps=1)
      # df_fc에 예측한 wl 결과값 추가하기
      df_fc.iloc[4464+i,0]=fc[0][0]

In [ ]:
# 예측값 입력할 df
df_fc_y1 = pd.concat([df_y1_pc_newTrain, df_y1_pc_newTest], axis=0)
df_fc_y2 = pd.concat([df_y2_pc_newTrain, df_y2_pc_newTest], axis=0)
df_fc_y3 = pd.concat([df_y3_pc_newTrain, df_y3_pc_newTest], axis=0)
df_fc_y4 = pd.concat([df_y4_pc_newTrain, df_y4_pc_newTest], axis=0)

# forecast_input df (< 10분 전 정답 데이터 사용)
df_tt_y1 = pd.concat([df_y1_pc_newTrain, df_y1_pc_dap], axis=0)
df_tt_y2 = pd.concat([df_y2_pc_newTrain, df_y2_pc_dap], axis=0)
df_tt_y3 = pd.concat([df_y3_pc_newTrain, df_y3_pc_dap], axis=0)
df_tt_y4 = pd.concat([df_y4_pc_newTrain, df_y4_pc_dap], axis=0)

In [ ]:
df_fc_y1.tail()

,wl_1018662,pc1,pc2,pc3,pc4,pc5
ymdhm,,,,,,
2022-07-18 23:10:00,-0.252626,0.123182,-0.172778,0.010886,-0.006156,-0.015705
2022-07-18 23:20:00,-0.252626,0.102676,-0.164035,0.009340,-0.006746,-0.022059
2022-07-18 23:30:00,-0.252626,0.086847,-0.149238,0.012111,-0.007474,-0.028260
2022-07-18 23:40:00,-0.252626,0.070589,-0.135303,0.014379,-0.008305,-0.035426
2022-07-18 23:50:00,-0.252626,0.058249,-0.116626,0.020054,-0.009060,-0.041096


In [ ]:
get_fc_ys(11, df_y1_pc_train, df_tt_y1, df_fc_y1)
get_fc_ys(11, df_y2_pc_train, df_tt_y2, df_fc_y2)
get_fc_ys(11, df_y3_pc_train, df_tt_y3, df_fc_y3)
get_fc_ys(11, df_y4_pc_train, df_tt_y4, df_fc_y4)
df_fc_y1.tail()

,wl_1018662,pc1,pc2,pc3,pc4,pc5
ymdhm,,,,,,
2022-07-18 23:10:00,0.029265,0.123182,-0.172778,0.010886,-0.006156,-0.015705
2022-07-18 23:20:00,0.029351,0.102676,-0.164035,0.009340,-0.006746,-0.022059
2022-07-18 23:30:00,0.029972,0.086847,-0.149238,0.012111,-0.007474,-0.028260
2022-07-18 23:40:00,0.044761,0.070589,-0.135303,0.014379,-0.008305,-0.035426
2022-07-18 23:50:00,0.056811,0.058249,-0.116626,0.020054,-0.009060,-0.041096


In [ ]:
# y_pred 데이터만
a=df_fc_y1.iloc[4464:,[0]]
b=df_fc_y2.iloc[4464:,[0]]
c=df_fc_y3.iloc[4464:,[0]]
d=df_fc_y4.iloc[4464:,[0]]

df_fc_sc_ys=a.merge(b, on='ymdhm')
df_fc_sc_ys=df_fc_sc_ys.merge(c, on='ymdhm')
df_fc_sc_ys=df_fc_sc_ys.merge(d, on='ymdhm')
df_fc_sc_ys

,wl_1018662,wl_1018680,wl_1018683,wl_1019630
ymdhm,,,,
2022-06-01 00:00:00,0.018435,0.033362,0.032664,0.046830
2022-06-01 00:10:00,0.019469,0.033384,0.033834,0.039227
2022-06-01 00:20:00,0.024677,0.033051,0.033916,0.036431
2022-06-01 00:30:00,0.029343,0.033421,0.041202,0.033421
2022-06-01 00:40:00,0.025892,0.039166,0.054590,0.035288
...,...,...,...,...
2022-07-18 23:10:00,0.029265,0.044770,0.051336,0.104646
2022-07-18 23:20:00,0.029351,0.055809,0.061807,0.109519
2022-07-18 23:30:00,0.029972,0.064175,0.067741,0.112426


In [ ]:
# 예측한 데이터
df_forecast_inv=y_scaler.inverse_transform(df_fc_sc_ys)
df_submission=pd.DataFrame(df_forecast_inv, columns=df_fc_sc_ys.columns, index=df_fc_sc_ys.index)
submission_inv_np=df_submission.values

# 정답 데이터
ys_dap_np=ys_dap.values

# RMSE
from sklearn.metrics import mean_squared_error
rmse=mean_squared_error(ys_dap_np, submission_inv_np)**0.5
print('RMSE score :',rmse)

# RMSE/R2
from sklearn.metrics import r2_score
def get_r2_score(pred,label):
    r2_list = []
    for i in range(4):
        p = pred[:,i]
        l = label[:,i]
        r2_s = r2_score(l,p)
        r2_list.append(r2_s)
    return np.mean(r2_list)

R2=get_r2_score(submission_inv_np,ys_dap_np)
print('RMSE/R2 score :', rmse/R2)

RMSE score : 3.332381401724188
RMSE/R2 score : 3.3381731868834894
